In [41]:
import json, requests, csv
import pandas as pd
import csv, re, time
from datetime import datetime
from textblob import TextBlob
import numpy as np
from bs4 import BeautifulSoup
from dateutil.parser import parse
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
import math
import concurrent.futures

In [42]:
# creation du fichier company

url_company = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/Company-2024-03-03.csv'
data_company = pd.read_csv(url_company)


# creation du fichier keyword

url_GoogleSearchKeyword = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/GoogleSearchKeyword-2024-03-03.csv'
data_GoogleSearchKeyword = pd.read_csv(url_GoogleSearchKeyword)

# creation du fichier googleresults

url_google_results = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/GoogleSearchResult-2024-03-03.csv'
data_google_results = pd.read_csv(url_google_results)

In [43]:
data_company.head()

,id,siren,name,brand_name,description,address,city,postal_code,country,phone,...,ape,founded_date,turnover_2022,gouv_searched,gouv_street,gouv_city,gouv_postal_code,gouv_country,gouv_lat,gouv_lng
0,164,847593225,AAA COIFFURE,AAA COIFFURE,NaN,11 RUE PAGANINI,NICE,6000,NaN,NaN,...,9602A,2019-01-15,33300.0,1.0,Rue Paganini,Nice,6000.0,NaN,43.702199,7.263926
1,203,331057406,ACA NEXIA,ACA NEXIA,NaN,31 RUE HENRI ROCHEFORT,PARIS,75017,NaN,NaN,...,6920Z,1984-09-01,18602980.0,1.0,Rue Henri Rochefort,Paris,75017.0,NaN,48.882511,2.308276
2,163,845298249,AEV AUTO ECOLE VEILLON,AUTO ECOLE VEILLON,NaN,11 AVENUE VILLERMONT,NICE,6000,NaN,NaN,...,8553Z,2019-01-09,244188.0,1.0,Avenue Villermont,Nice,6000.0,NaN,43.709300,7.264297
3,527,316473305,AGRI-POLE,AGRI-POLE,NaN,RUE SAINT VICTOR,VILLEFRANCHE-DE-ROUERGUE,34420,NaN,NaN,...,4661Z,1979-01-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,310,349638395,AIR CORSICA,CCM AIRLINES,NaN,CAMPO DELL ORO,AJACCIO,20090,NaN,NaN,...,5110Z,1990-01-01,130850508.0,1.0,Campo Dell Oro,Ajaccio,20090.0,NaN,41.929606,8.804378


In [44]:
data_company[data_company['id'] == 320]

,id,siren,name,brand_name,description,address,city,postal_code,country,phone,...,ape,founded_date,turnover_2022,gouv_searched,gouv_street,gouv_city,gouv_postal_code,gouv_country,gouv_lat,gouv_lng
268,320,377550249,WAVESTONE,WAVESTONE,NaN,100 TERRASSE BOIELDIEU,PUTEAUX,92800,NaN,NaN,...,6202A,1990-04-01,377647000.0,1.0,Terrasse Boieldieu,Puteaux,92800.0,NaN,48.889287,2.24075


In [45]:
data_GoogleSearchKeyword.head()

,id,active,keyword,news
0,116,1,"""actualités""",0
1,149,1,"""annonce""",0
2,154,1,"""controverses"" OR ""problème"" OR ""litige""",0
3,153,1,"""événement"" OR ""séminaire"" OR ""atelier""",0
4,151,1,"""lancement de produit"" OR ""nouveau service""",0


In [46]:
data_google_results.head()

,id,search,company,google_results,scraped_date,scraped
0,1999,116,545,NaN,2024-03-03 07:02:38,NaN
1,372,116,309,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:31,1.0
2,371,116,491,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:27,1.0
3,370,116,506,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:23,1.0
4,369,116,320,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:24,1.0


In [7]:
company_list = []
SIRET_list = []

# Itérer sur chaque ligne du DataFrame
for index, occurrence in data_company.iterrows():
    company = occurrence['id']  # Assurez-vous que 'id' est le nom correct de la colonne
    SIRET = occurrence['siret']  # Assurez-vous que 'siret' est le nom correct de la colonne
    company_list.append(company)
    SIRET_list.append(SIRET)

Id_SIRET = dict(zip(company_list, SIRET_list))

In [8]:
Pk_search_list = []
keyword_list = []

# Itérer sur chaque ligne du DataFrame
for index, occurrence in data_GoogleSearchKeyword.iterrows():
    Pk_search = occurrence['id']
    keyword = occurrence['keyword']
    Pk_search_list.append(Pk_search)
    keyword_list.append(keyword)

Search_keyword = dict(zip(Pk_search_list, keyword_list))

In [9]:
Search_keyword

{116: '"actualités"',
 149: '"annonce"',
 154: '"controverses" OR "problème" OR "litige"',
 153: '"événement" OR "séminaire" OR "atelier"',
 151: '"lancement de produit" OR "nouveau service"',
 152: '"nomination" OR "départ" OR "nouveau dirigeant"',
 150: '"partenariat" OR "collaboration"'}

In [10]:
data_modifie = []

# Parcourir chaque élément du google search, créer un dictionnaire id (company) / SIRET / keyword / contenu

for index, element in data_google_results.iterrows():
    nouvel_element = {}
    for cle, valeur in element.items():
        if cle == 'company':
            nouvel_element['SIRET'] = Id_SIRET[element['company']] # affecter le SIRET en fonction de la table company (Id)
        elif cle == 'search':
            nouvel_element['keyword'] = Search_keyword[element['search']]
        else:
            nouvel_element[cle] = valeur
    data_modifie.append(nouvel_element)

In [11]:
data_modifie = pd.DataFrame(data_modifie)

In [12]:
data_modifie

,id,keyword,SIRET,google_results,scraped_date,scraped
0,1999,"""actualités""",408627354 00124,NaN,2024-03-03 07:02:38,NaN
1,372,"""actualités""",34899155500353,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:31,1.0
2,371,"""actualités""",30649449300050,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:27,1.0
3,370,"""actualités""",31124863700804,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:23,1.0
4,369,"""actualités""",37755024900041,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:24,1.0
...,...,...,...,...,...,...
1899,1732,"""partenariat"" OR ""collaboration""",34963839500021,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:31,1.0
1900,1731,"""partenariat"" OR ""collaboration""",31647330500035,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:30,1.0
1901,1730,"""partenariat"" OR ""collaboration""",84529824900028,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:47,1.0
1902,1729,"""partenariat"" OR ""collaboration""",33105740600067,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:37,1.0


In [13]:
def extraire_donnees(json_str, siret, keyword):
    try:
        # Assurez-vous que json_str est une chaîne
        if not isinstance(json_str, str):
            print(f"Erreur : json_str n'est pas une chaîne. SIRET: {siret}, Keyword: {keyword}")
            return []

        # Charger la chaîne JSON en dictionnaire
        data = json.loads(json_str)
        results = []
        
        # Itération sur les résultats organiques
        for result in data.get('organic_results', []):
            res_dict = {
                'siret': siret,
                'keyword': keyword,
                'url': result.get('url', ''),
                'title': result.get('title', ''),
                'domain': result.get('domain', ''),
                'description': result.get('description', ''),
                'displayed_url': result.get('displayed_url', '')
            }
            if isinstance(result.get('sitelinks'), dict):
                res_dict['sitelinks'] = result.get('sitelinks', {}).get('expanded', [])
            else:
                res_dict['sitelinks'] = []
            results.append(res_dict)
        
        return results
    except Exception as e:
        print(f"Erreur lors de l'extraction: {e}, siret: {siret}, keyword: {keyword}")
        return []
    
resultats_extraits = []

# Itérer sur chaque ligne du DataFrame data_modifie
for idx, row in data_modifie.iterrows():
    # Extraire les données organiques, le siret et le mot-clé pour chaque ligne
    resultats_temp = extraire_donnees(row['google_results'], row['SIRET'], row['keyword'])
    # Ajouter les résultats à la liste principale
    resultats_extraits.extend(resultats_temp)

# Convertir la liste des résultats extraits en un nouveau DataFrame
detail_google_search = pd.DataFrame(resultats_extraits)

Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "actualités"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "annonce"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "controverses" OR "problème" OR "litige"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "événement" OR "séminaire" OR "atelier"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "lancement de produit" OR "nouveau service"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "nomination" OR "départ" OR "nouveau dirigeant"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "partenariat" OR "collaboration"


In [14]:
detail_google_search

,siret,keyword,url,title,domain,description,displayed_url,sitelinks
0,34899155500353,"""actualités""",https://www.nicematin.com/conso-shopping/le-pl...,Le plus grand Zara de la région Sud a ouvert c...,www.nicematin.com,"May 4, 2023 — Nice-Matin. Se connecter · Abonn...",https://www.nicematin.com › le-plu...,[]
1,34899155500353,"""actualités""",https://nicepresse.com/le-plus-grand-zara-de-t...,Le plus grand Zara de tout le Sud-Est bientôt ...,nicepresse.com,"Oct 31, 2022 — Nice-Presse : actualités et inf...",https://nicepresse.com › le-plus-gra...,[]
2,34899155500353,"""actualités""",https://actu.fr/provence-alpes-cote-d-azur/nic...,Nice. Pourquoi y avait-il une grosse file d'at...,actu.fr,"Nov 25, 2022 — Dernières actualités. Actu Nice...",https://actu.fr › nice_06088 › nice-...,[]
3,34899155500353,"""actualités""",https://actu.fr/provence-alpes-cote-d-azur/sai...,"Près de Nice, CAP 3000 annonce de nouvelles bo...",actu.fr,"Jan 19, 2023 — L'un des plus grands Zara de Fr...",https://actu.fr › pres-de-nice-cap-30...,[]
4,34899155500353,"""actualités""",https://www.lefigaro.fr/tag/zara,Zara : dernières actualités et vidéos,www.lefigaro.fr,Les dernières actualités sur Zara.,https://www.lefigaro.fr › tag › zara,[]
...,...,...,...,...,...,...,...,...
13930,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=9QIdYbTIKQAC&...,Le Mémorial diplomatique,books.google.fr,... AAA Vient de Paraitre chez PAUL OLLENDORFF...,https://books.google.fr › books,[]
13931,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=PbKXBAAAQBAJ&...,Michelin Green Guide France - Google Books Result,books.google.fr,... collaboration with Coysevox and Tuby. In t...,https://books.google.fr › books,[{'link': '/search?num=100&sca_esv=566872717&g...
13932,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=CjiXEAAAQBAJ&...,Sekiro: La seconde vie des Souls - Google Book...,books.google.fr,... partenariat avec Activision en surprend – ...,https://books.google.fr › books,[{'link': '/search?num=100&sca_esv=566872717&g...
13933,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=qZZ-CwAAQBAJ&...,Robert Trent Jones Golf Trail: Its History and...,books.google.fr,... collaboration between the City of Montgome...,https://books.google.fr › books,[{'link': '/search?num=100&sca_esv=566872717&g...


In [15]:
# Ajoutez des colonnes pour le résumé et la date dans le DataFrame
detail_google_search['resume'] = pd.NA
detail_google_search['date'] = pd.NA

In [20]:

# Découper le DataFrame en sous-ensembles en tenant compte des changements de SIRET
start_idx = 0  # Index de départ pour la tranche actuelle
for i in range(taille_chunk, len(detail_google_search_sorted) + taille_chunk, taille_chunk):
    # Trouver la fin de la tranche en s'assurant que tous les enregistrements d'un SIRET sont ensemble
    # Si on n'est pas à la fin du DataFrame
    if i < len(detail_google_search_sorted):
        while (i < len(detail_google_search_sorted) and 
               detail_google_search_sorted.iloc[i - 1]['siret'] == detail_google_search_sorted.iloc[i]['siret']):
            i += 1  # Déplacer l'index de fin jusqu'à ce qu'on trouve un nouveau SIRET
    
    # Découper le DataFrame jusqu'à cet indice
    dfs.append(detail_google_search_sorted[start_idx:i])
    
    # Mettre à jour l'index de départ pour la prochaine tranche
    start_idx = i

In [39]:
nombre_tranches = len(dfs)-1
print(f"Nombre de tranches créées : {nombre_tranches}")
print(f"tranche 1 : {len(dfs[1])}")
print(f"tranche 2 : {len(dfs[2])}")
print(f"tranche 3 : {len(dfs[3])}")
print(f"tranche 4 : {len(dfs[4])}")
print(f"tranche 5 : {len(dfs[5])}")
print(f"total: {len(dfs[1])+len(dfs[2])+len(dfs[3])+len(dfs[4])+len(dfs[5])}")


Nombre de tranches créées : 5
tranche 1 : 2795
tranche 2 : 2932
tranche 3 : 2639
tranche 4 : 2784
tranche 5 : 2785
total: 13935


In [167]:
def scraper_et_resume(url):
    attempts = 0
    while attempts < 1:  # une tentative
        try:
            # Effectuer une requête GET sur le site
            response = requests.get(url, timeout=10)  # Augmentez le timeout si nécessaire
            # Créer un objet BeautifulSoup pour analyser le HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire le texte souhaité, par exemple, tous les paragraphes
            text = ' '.join(p.text for p in soup.find_all('p'))

            # Recherche de la date dans les métadonnées
            meta_date = soup.find('meta', {'property': 'article:published_time'})
            if meta_date:
                date = meta_date['content']
            else:
                # Ajoutez le nouveau motif pour les dates avec mois en lettres
                date_patterns = [
                    r'\b\d{4}-\d{2}-\d{2}\b',
                    r'\b\d{2}/\d{2}/\d{4}\b',
                    r'\b\d{2}-\d{2}-\d{4}\b',
                    # Motif pour le format "14 octobre 2023" ou "14 Octobre 2023"
                    r'\b\d{1,2}\s(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s\d{4}\b',
                    r'\b\d{1,2}\s(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}\b'
                ]
                date = 'Date non trouvée'
                for pattern in date_patterns:
                    match = re.search(pattern, text, re.IGNORECASE)  # Utiliser IGNORECASE pour ne pas se soucier de la casse
                    if match:
                        date = match.group()
                        break

            # Création d'un résumé du texte
            parser = PlaintextParser.from_string(text, Tokenizer('french'))
            summarizer = Summarizer()
            resume_text = ' '.join(str(sentence) for sentence in summarizer(parser.document, 10))
            return resume_text, date
        except requests.exceptions.RequestException as e:
            attempts += 1
            time.sleep(5)  # Délai entre les tentatives
    
    # Retourner None si toutes les tentatives échouent
    return None, None

def analyze_sentiment(sentence):
    analysis = TextBlob(sentence)
    if analysis.sentiment.polarity > 0:
        return 'Positif'
    elif analysis.sentiment.polarity == 0:
        return 'Neutre'
    else:
        return 'Négatif'
    
# Fonction pour convertir les sentiments en poids
def sentiment_to_weight(sentiment):
    weights = {'Positif': 1, 'Neutre': 0, 'Négatif': -1}
    return weights.get(sentiment, 0)  # Retourne 0 si le sentiment n'est pas reconnu

In [ ]:
def traiter_dataframe(df):
    # fonction de parralelisation
    
    print(f"Traitement du DataFrame de taille {len(df)}")
    
    # Réinitialise l'index du DataFrame pour une itération propre (df_index(drop=True) devrait être df.reset_index(drop=True) 
    # pour fonctionner correctement).
    df = df_index(drop=True)
    
    # Calcul du total des lignes et du point de contrôle pour suivre la progression du traitement, en découpant le DataFrame en dix segments (10 % chacun) 
    # pour les rapports d'avancement.
    total_lignes = len(df)
    print(total_lignes)
    pourcent = total_lignes / 10
    compteur=0
    
    # Itère sur chaque ligne du DataFrame pour exécuter scraper_et_resume sur l'URL de chaque ligne. 
    # Met à jour le DataFrame avec les nouveaux résumés et dates récupérés, à condition que le résumé ne soit pas vide.
    for index, row in df.iterrows():
    try:
        resume, date = scraper_et_resume(row['url'])
        # Vérifiez si le résumé est vide avant de mettre à jour le DataFrame
        if resume:  # Si 'resume' n'est pas vide
            df.at[index, 'resume'] = resume
            df.at[index, 'date'] = date
        else:  
            print(f"Contenu vide pour l'URL {row['url']} à la ligne {index}, pas de mise à jour.")
            continue  # Passe à l'itération suivante sans mettre à jour les valeurs dans le DataFrame
    # Gère les exceptions qui peuvent survenir pendant le processus de scraping, permettant à la boucle de continuer même en cas d'erreur.        
    except Exception as e:
        print(f"Erreur lors du traitement de la ligne {index}: {e}")
        continue  # Passe à l'itération suivante en cas d'erreur
    
    print(f"Avancement: {math.ceil((compteur / total_lignes) * 100)}% terminé.")
    compteur+=1
    
    # Supprime les lignes où la colonne 'date' est NaN après le scraping pour assurer l'intégrité des données et remplace 
    # les chaînes "Date non trouvée" par NaN, puis convertit les chaînes de date restantes en objets datetime, 
    #en gérant les erreurs et en supprimant le fuseau horaire pour uniformité.
    df = df.dropna(subset=['date'])

    # Remplacer "Date non trouvée" par NaN puis convertir les chaînes de date en objets datetime
    df['date'] = df['date'].replace('Date non trouvée', pd.NA)
    df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True)

    # Obtenir la date du jour en UTC pour garder la cohérence avec les autres dates
    date_du_jour = pd.Timestamp('now', tz='UTC')

    # Remplacer les valeurs NaN par la date du jour en format UTC
    df['date'] = df['date'].fillna(date_du_jour)
    df['date'] = df['date'].dt.tz_localize(None)

    # Extraire l'année de la colonne de date
    df['year'] = df['date'].dt.year
    
    # Appliquer la fonction analyze_sentiment à chaque résumé dans le DataFrame
    df['sentiment'] = df['resume'].apply(analyze_sentiment)
    
    # Convertir les dates en datetime et extraire l'année
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['year'] = df['date'].dt.year

    # Appliquer la fonction pour convertir les sentiments en poids
    df['sentiment_weight'] = df['sentiment'].apply(sentiment_to_weight)

    # Calculer la moyenne pondérée des sentiments par année
    weighted_average_by_year = df.groupby('year')['sentiment_weight'].mean()
    
    # Calculer la moyenne pondérée des sentiments par année et par SIRET
    grouped = df.groupby(['siret', 'year'])['sentiment_weight'].mean().reset_index()

    # Pivoter pour obtenir les années en colonnes et les SIRETs en lignes
    pivot_table = grouped.pivot(index='siret', columns='year', values='sentiment_weight')

    # Renommer les colonnes pour le format souhaité
    pivot_table.columns = [f'sentiment weight an{year}' for year in pivot_table.columns]

    # Réinitialiser l'index pour obtenir 'siret' comme une colonne
    final_output = pivot_table.reset_index()

    # Afficher le DataFrame final
    print(final_output)
    
        return df, final_output

In [ ]:
# Utilisation de ThreadPoolExecutor pour paralléliser le traitement

dfs_adaptes = []  # Liste pour stocker tous les DataFrames adaptés
dfs_final_output = []  # Liste pour stocker tous les DataFrames final_output

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Planifier l'exécution de la fonction de traitement pour chaque sous-DataFrame
    futures = [executor.submit(traiter_dataframe, df) for df in dfs]

    # Attendre que tous les futurs soient terminés et recueillir les résultats
    for future in concurrent.futures.as_completed(futures):
        df_adapte, final_out = future.result()
        dfs_adaptes.append(df_adapte)
        dfs_final_output.append(final_out)

# Fusionner tous les DataFrames adaptés en un seul
data_google_search = pd.concat(dfs_adaptes)

# Fusionner tous les final_output en un seul
global_final_output = pd.concat(dfs_final_output)


In [ ]:
# Obtenir la date actuelle
date_actuelle = datetime.now().strftime('%Y-%m-%d')  # Formate la date comme "AAAA-MM-JJ"

# Créer le nom du fichier avec la date
nom_fichier = f"search_Google_{date_actuelle}_actu_resumé.csv"

# Exporter le DataFrame en CSV
data_google_search.to_csv(nom_fichier, index=False)

In [ ]:
#referentiel base source
source_entreprise=pd.read_csv('base_source_enrichie.csv', encoding='utf-8', sep=',', on_bad_lines='skip')
source_entreprise.shape

In [ ]:
# Fusionner les DataFrames  base enrichie et final output sur la colonne 'siret'
df_enrichie = pd.merge(source_entreprise, global_final_output, on='siret', how='left')

In [ ]:
# Exporter le DataFrame en CSV
nom-fichier='base_source_enrichie.csv'
df_enrichie.to_csv(nom_fichier, index=False)